In [113]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import random
import sys
import io


#Importing the Data and TwitterBots Credentials

In [135]:
from credentials import *

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Open text file text_corpus.txt  for reading. 
#20000 Lines from the corpus provided with the Assignment was used 
filename = "text_corpus.txt" 
my_file = open(filename).read()


file_lines = my_file.lower()



In [137]:
chars = sorted(list(set(file_lines)))
print('total chars:', len(chars))
print(chars)

total chars: 37
['\n', ' ', '!', '&', "'", ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [138]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [139]:

n_chars = len(file_lines)
n_vocab = len(chars)
print ("Total Characters: %d"% n_chars)
print ("Total Vocab: %d"% n_vocab)


Total Characters: 566475
Total Vocab: 37


In [140]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 120
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = file_lines[i:i + seq_length]
    seq_out = file_lines[i + seq_length]
    dataX.append([char_indices[char] for char in seq_in])
    dataY.append(char_indices[seq_out])
    
n_patterns = len(dataX)
print ("Total Patterns: %d"% n_patterns)


Total Patterns: 566355


In [141]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# training LSTM 

In [145]:
# define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=3, batch_size=12)

Epoch 1/3
566355/566355 [==============================] - 3670s 6ms/step - loss: 2.7113
Epoch 2/3
566355/566355 [==============================] - 3582s 6ms/step - loss: 2.4777
Epoch 3/3
566355/566355 [==============================] - 2903s 5ms/step - loss: 2.3882


In [153]:
#function for generating text
def tweet(dataX,model):
    start = np.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    #print((pattern))
    #print ("********************************")
    #print("Seed:")
    #print("\"", ''.join([indices_char[value] for value in pattern]), "\"")
    #print("*****")
    # generate characters
    seq_out=[]

    # generate characters
    for i in range(100):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = indices_char[index]
        seq_in = [indices_char[value] for value in pattern]
        #sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    print ("\nDone.")
    seq_out=''.join([indices_char[value] for value in pattern])
    return seq_out    

In [166]:
from time import sleep
# Write a tweet to push to our Twitter account

for item in range(20):
    try:
        print(item)
        output=tweet(dataX,model)
        api.update_status(status=output)
    except tweepy.TweepError as e:
        print(e.reason)
    sleep(360)